In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from numpy import sin, cos, arccos, pi, round

from sklearn import neighbors
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Leitura e tratamento inicial dos dados

Lendo a CSV e removendo todos os dados com valores de Tp_est iguais a zero

In [22]:
df_raw_data = pd.read_csv('/content/drive/MyDrive/BCC - UFPR/Semestres /9 - 2023-2/Aprendizado de Maquina/Lab2/Dados_Radar_Estacao_Completo_2018_2022.csv')
df_raw_data.drop(df_raw_data[df_raw_data['Tp_est'] == 0.0].index, inplace=True);

Remoção inicial de colunas da base geral (2018 a 2022). A colunas removidas foram:
- Unnamed: 0 pois é a coluna de ids
- latitude e longitude pois possuem a mesma informação que as colunas lat e lon;
- distancia: Não agrega valor ao modelo. (*)

In [23]:
df_raw_data.drop(['Unnamed: 0', 'latitude', 'longitude', 'distancia'], axis=1, inplace=True)

Separando o raw data entre treinamento e teste. Anos de 2018 a 2021 para treinamento e 2022 para teste.


In [24]:
raw_train_test_group = df_raw_data.groupby(df_raw_data['time'].str.contains('2022'))

df_raw_train = raw_train_test_group.get_group(False).copy()
df_raw_test = raw_train_test_group.get_group(True).copy()

Separando os dados de treinamento, selecionando uma porção dos dados para validação

In [25]:
dates = ['2018-01', '2018-02']
# dates = ['2018-01']

raw_train_validation_group = df_raw_train.groupby(df_raw_train['time'].str.contains('|'.join(dates)))
df_raw_train = raw_train_validation_group.get_group(False).copy()
df_raw_validation = raw_train_validation_group.get_group(True).copy()


Apos essa separação, possuimos 3 bases:
- df_raw_train: Base para treinamento
- df_raw_validation: Base para validação
- df_raw_test: Base para teste

Remoção das colunas elevation e sweep pois não possuiam valor agregado na base

Para verificar que as colunas elevation e sweep não possuiam valor agregado, foi utilizado a função describe do pandas. Com esta função, foi possível verificar que ambas possuiam média, minimo e maximo identicos, além de um desvio padrão igual a 0, ou seja, todas as linhas possuiam o mesmo valor.

In [26]:
print(df_raw_train.describe()['elevation'], end="\n\n")
print(df_raw_train.describe()['sweep'])

df_raw_train.drop(['elevation', 'sweep'], axis=1, inplace=True)

count    83083.0
mean         0.5
std          0.0
min          0.5
25%          0.5
50%          0.5
75%          0.5
max          0.5
Name: elevation, dtype: float64

count    83083.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: sweep, dtype: float64


### Etapa de clusterização dos dados por estação

Criando 2 DataFrames onde um deles armazena as informações das colunas X e Y da base e o outro armazena as colunas LAT e LON


In [7]:
grouped_by_est = df_raw_train.groupby(['Est'])

df_xy = pd.DataFrame(columns=['est','x', 'y'])
df_lat_lon = pd.DataFrame(columns=['est', 'lat', 'lon'])
for est in grouped_by_est.groups.keys():
    group = grouped_by_est.get_group(est)

    list_xy = [est, group['x'].mean(), group['y'].mean()]
    df_xy = pd.concat([pd.DataFrame([list_xy], columns=df_xy.columns), df_xy], ignore_index=True)

    list_lat_lon = [est, group['lat'].mean(), group['lon'].mean()]
    df_lat_lon = pd.concat([pd.DataFrame([list_lat_lon], columns=df_lat_lon.columns), df_lat_lon], ignore_index=True)

Verificando que os valores X e Y fazer o mesmo papel que os valores de LAT e LON para o calcula da distância entre as estações

In [ ]:
def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

theta = df_lat_lon['lon'][0] - df_lat_lon['lon'][1]
distance = 60 * 1.1515 * rad2deg(
    arccos(
        (sin(deg2rad(df_lat_lon['lat'][0])) * sin(deg2rad(df_lat_lon['lat'][1]))) +
        (cos(deg2rad(df_lat_lon['lat'][0])) * cos(deg2rad(df_lat_lon['lat'][1])) * cos(deg2rad(theta)))
    )
)
distance = round(distance * 1.609344, 5)
print(distance)

# Distância euclidina por meio do numpyu
point1 = np.array((df_xy['x'][0], df_xy['y'][0]))
point2 = np.array((df_xy['x'][1], df_xy['y'][1]))
distance = np.linalg.norm(point1 - point2)
print(round(distance / 1000, 5))


Plotando um gráfico das posições das estações para ter um auxilio visual na clusterização


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for data in df_lat_lon.iterrows():
    ax.scatter(data[1][1], data[1][2], label=f'{data[1][0]} - {data[0]}')

colormap = plt.cm.gist_ncar
colorst = [colormap(i) for i in np.linspace(0.1, 0.9,len(ax.collections))]
for t, j1 in enumerate(ax.collections):
    j1.set_color(colorst[t])

for data in df_lat_lon.iterrows():
    ax.annotate(data[0], (data[1][1], data[1][2]))

ax.legend(bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes, fontsize='small')
plt.show()

# fig = plt.figure()
# ax = fig.add_subplot(111)
# for data in df_xy.iterrows():
#     ax.scatter(data[1][1], data[1][2], label=f'{data[1][0]} - {data[0]}')

# colormap = plt.cm.gist_ncar
# colorst = [colormap(i) for i in np.linspace(0.1, 0.9,len(ax.collections))]
# for t, j1 in enumerate(ax.collections):
#     j1.set_color(colorst[t])

# for data in df_xy.iterrows():
#     ax.annotate(data[0], (data[1][1], data[1][2]))

# ax.legend(bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes, fontsize='small')
# plt.show()

Calculando a distancia de cada estação entre si para ter um embasamento numérico na decisão da clusterização

In [ ]:
def euclidian_distance(x1, y1, x2, y2):
    p1 = np.array((x1, y1))
    p2 = np.array((x2, y2))
    return np.linalg.norm(p1 - p2)

est_distances = []
for est in df_xy.iterrows():
    distances = []
    for row in df_xy.iterrows():
        if est[1][0] != row[1][0]:
            distance = round((euclidian_distance(est[1][1], est[1][2], row[1][1], row[1][2]) / 1000), 3)
            distances.append((distance, row[1][0]))
    distances = sorted(distances)
    distances_sum = sum(i[0] for i in distances[:6])
    est_distances.append((distances_sum, est[1][0], distances))

est_distances = sorted(est_distances)

for est in est_distances:
    print(f'Estação: {est[1]} - Soma: {est[0]} - Estações: {est[2][:6]}')

Remoção das colunas X, Y, Z, LAT, LON, ALT, ja que a partir de agora, a clusterização vai trazer a informação de posicionamento

In [16]:
df_raw_train.drop(['x', 'y', 'z', 'lat', 'lon', 'alt'], axis=1, inplace=True)

Após alguns testes e tentativas de divisão, a clusterização final ficou como:


In [27]:
grouped_by_est = df_raw_train.groupby(['Est'])
dict_df_train = {'df_train' : {}, 'for_input': {}}

groups = [
    ['Pato_Branco', 'Laranjeiras_do_Sul', 'Segredo', 'Derivacao_do_Rio_Jordao', 'Coronel_Domingos_Soares', 'Solais_Novo'],
    ['Cascavel', 'Baixo_Iguacu', 'Salto_Caxias', 'Reservatorio_Salto_Caxias', 'Boa_Vista_da_Aparecida', 'Porto_Santo_Antonio', 'Aguas_do_Vere', 'Bela_Vista_Jusante'],
    ['Foz_do_Iguacu_-_Itaipu', 'Santa_Helena', 'Guaira', 'Palotina', 'Toledo', 'Assis_Chateaubriand', 'Altonia'],
    ['Loanda', 'Paranavai', 'Campo_Mourao', 'Umuarama', 'Ubirata', 'Porto_Formosa']
]

for group in groups:
    df_groupped = grouped_by_est.get_group(group[0])
    for index in range(1, len(group)):
        df_groupped = pd.concat([grouped_by_est.get_group(group[index]), df_groupped], ignore_index=True)
    df_groupped.drop(['Est'], axis=1, inplace=True)

    label = ' - '.join(map(str, group))
    dict_df_train['df_train'][label] = df_groupped


for group in dict_df_train['df_train']:
    df_groupped = dict_df_train["df_train"][group].dropna()
    columns = list(df_groupped.head())

    dict_df_train['for_input'][group] = {}
    for row in df_groupped.iterrows():
        timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

        row_info = {}
        for col in columns:
            row_info[col] = row[1][col]

        dict_df_train['for_input'][group][timestamp] = row_info


### Tratamento dos dados faltantes

O dicionário 'for_input' produzido acima possui as seguintes características:
- Dividido em grupos da mesma forma que os datasets de treinamento;
- Possui apenas as linhas que possuiam todos os dados dos sensores dos datasets de treinamento;
- Foi organizado por meio de dicionários internos com a chave sendo o timestamp do dado para facilitar a busca.

Com este dicionário, o tratamento dos dados faltantes foi realizado de forma clusterizada, ou seja, on inputs foram realizados apenas com dados do própio grupo

In [28]:
def get_data(row, input_values):
    timestamp = datetime.strptime(row["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    data = None
    min_distance = math.inf
    for i in input_values:
        distance = abs(timestamp - i)
        if (distance < min_distance):
            min_distance = distance
            data = input_values[i]

    return data

def df_input_data(df_group, input_values):
    columns = list(df_group.head())
    for row in df_group.iterrows():
        replace_data = get_data(row[1], input_values)
        for col in columns:
            if str(row[1][col]) == 'nan':
                df_group.loc[row[0], col] = replace_data[col]

for group in dict_df_train['df_train']:
    df_group = dict_df_train['df_train'][group]
    input_values = dict_df_train['for_input'][group]
    df_input_data(df_group, input_values)
    dict_df_train['df_train'][group].drop(['time'], axis=1, inplace=True)

### Preparação da base de validação

Removendo as colunas da base de validação que foram removidas da base de treinamento

In [29]:
# df_validation = df_raw_validation.drop(['elevation', 'sweep', 'x', 'y', 'z', 'lat', 'lon', 'alt'], axis=1)
df_validation = df_raw_validation.drop(['elevation', 'sweep'], axis=1)

Realizando o tratamento de dados faltantes na base de validação. O input foi feito da seguinte maneira:

Para realizar o tratamento, primeiro, para cada linha, é buscado o grupo a qual ela pertence, ou seja, de qual grupo de estações o dado veio. Após isso, apenas a base de treinamento é utilizada para o tratamento, sem a influência dos dados da base de validação.

In [30]:
def get_group(est, input_groups):
    for group in input_groups:
        if est in group: return input_groups[group]

columns = list(df_validation.head())
for row in df_validation.iterrows():
    input_values = get_group(row[1]['Est'], dict_df_train['for_input'])
    replace_data = get_data(row[1], input_values)
    for col in columns:
        if str(row[1][col]) == 'nan':
            df_validation.loc[row[0], col] = replace_data[col]

df_validation.drop(['Est', 'time'], axis=1, inplace=True)

### Validação dos modelos de regressão

To-do: Utilizar a base de validação para definir o modelo que vou usar

In [31]:
def get_model(model):
    match model:
        case 'knn':
            return neighbors.KNeighborsRegressor(n_neighbors=100)
        case 'svr':
            return svm.SVR()
        case 'linear_regression':
            return linear_model.LinearRegression()
        case 'tree_regression':
            return tree.DecisionTreeRegressor()

models = ['knn', 'svr', 'linear_regression', 'tree_regression']

validate_list = np.array(df_validation.values.tolist())
x_validate = validate_list[:, :-1]
y_validate = validate_list[:, -1]

preds = {'knn': [], 'svr': [], 'linear_regression': [], 'tree_regression': []}
for group in dict_df_train['df_train']:
    train_list = np.array(dict_df_train['df_train'][group].values.tolist())
    x_train = train_list[:, :-1]
    y_train = train_list[:, -1]

    for model_type in models:
        model = get_model(model_type)
        model.fit(x_train, y_train)

        pred = model.predict(x_validate)
        preds[model_type].append(pred)

avarages = {'knn': [], 'svr': [], 'linear_regression': [], 'tree_regression': []}
for model_pred in preds:
    avarage = preds[model_pred][0]
    for index in range(1, len(preds[model_pred])):
        avarage = [a + b for a, b in zip(avarage, preds[model_pred][index])]
    avarage = [x / len(preds[model_pred]) for x in avarage]

    avarages[model_pred] = avarage

results = {'mse': [], 'mae': []}
for avarage in avarages:
    mse = mean_squared_error(y_validate, avarages[avarage])
    mae = mean_absolute_error(y_validate, avarages[avarage])
    results['mse'].append(mse)
    results['mae'].append(mae)

    print(f'Modelo: {avarage} - MSE: {mse} - MAE: {mae}')

Modelo: knn - MSE: 6.043251417520959 - MAE: 1.257230230966638
Modelo: svr - MSE: 6.721039461352233 - MAE: 1.014354361004017
Modelo: linear_regression - MSE: 259.262143943076 - MAE: 11.659095839978125
Modelo: tree_regression - MSE: 8.958345470487597 - MAE: 1.9542386655260908


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for index, model in enumerate(models):
    ax.scatter(results['mae'][index], results['mse'][index], label=f'mse: {results["mse"][index]} - mae: {results["mae"][index]}')

for index, model in enumerate(models):
    ax.annotate(model, (results['mae'][index], results['mse'][index]))

ax.legend(bbox_to_anchor=(1.2, 1), bbox_transform=ax.transAxes, fontsize='small')
plt.show()